In [16]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [17]:
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229]),
])

In [18]:
class EggShellDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.data_frame.iloc[idx, 0]
        img_path = f"{self.root_dir}/{img_name}"
        image = Image.open(img_name).convert('L')
        measurements = self.data_frame.iloc[idx, 1:].values.astype('float')
        
        sample = {'image': image, 'measurements': measurements}

        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample

In [19]:
train_dataset = EggShellDataset(csv_file="imagens/Medidas ovos treino.csv", root_dir="imagens/", transform=image_transform)
test_dataset = EggShellDataset(csv_file="imagens/Medidas ovos teste.csv", root_dir="imagens/", transform=image_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [20]:
class CombinedModel(nn.Module):
    def __init__(self, num_tabular_features, num_outputs):
        super(CombinedModel, self).__init__()
        self.convnext = models.convnext_small(pretrained=True)
        self.convnext.classifier[2] = nn.Identity()

        self.tabular_net = nn.Sequential(
            nn.Linear(num_tabular_features, 128),
            nn.ReLU(),
            nn.Linear(128,128),
            nn.ReLU()
        )

        self.combined_net = nn.Sequential(
            nn.Linear(768 + 128, 256),
            nn.ReLU(),
            nn.Linear(256, num_outputs)
        )

    def forward(self, image, tabular_data):
        image_features = self.convnext(image)
        tabular_features = self.tabular_net(tabular_data)
        combined_features = torch.cat((image_features, tabular_features), dim=1)
        output = self.combined_net(combined_features)

        return output
    
    

In [21]:
num_tabular_features = train_dataset.data_frame.shape[1] - 1  # Número de colunas na planilha, exceto a coluna da imagem
num_outputs = 1  # Número de medidas a serem previstas

In [22]:
model = CombinedModel(num_tabular_features, num_outputs)

# Função de perda e otimizador
criterion = nn.MSELoss()  # Usar MSELoss para regressão
optimizer = optim.Adam(model.parameters(), lr=0.001)


c:\Users\camil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\camil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Small_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [25]:
num_epochs = 10  # Ajuste conforme necessário
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        images = batch['image']
        tabular_data = batch['measurements']
        
        # Forward pass
        outputs = model(images, tabular_data)
        loss = criterion(outputs, tabular_data)  # Ajuste conforme necessário

        # Backward pass e otimização
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Avaliação no conjunto de teste
    model.eval()
    with torch.no_grad():
        test_loss = 0
        for batch in test_loader:
            images = batch['image']
            tabular_data = batch['measurements']
            
            outputs = model(images, tabular_data)
            loss = criterion(outputs, tabular_data)
            test_loss += loss.item()
        
        test_loss /= len(test_loader)
        print(f'Test Loss after Epoch [{epoch+1}/{num_epochs}]: {test_loss:.4f}')

FileNotFoundError: [Errno 2] No such file or directory: 'imagens/train/ovo60'